# Predicción del Rendimiento Estudiantil con Árboles de Decisión

***Presentado por:***

Jean Giraldo

Mónica Cortés

Jhoimar Silva

Wendy Yepez

## **Objetivo del Estudio**
Este notebook implementa un modelo de clasificación basado en árboles de decisión para predecir si un estudiante será **aprobado (1)** o **reprobado (0)** en función de características académicas, demográficas y socioeconómicas.

## **Variables del Dataset**
El conjunto de datos contiene 1,044 observaciones con 17 variables:

### **Variable Objetivo:**
- approved: Aprobación del estudiante (0=No, 1=Sí)

## **Metodología**
1. Preprocesamiento de datos (normalización y codificación)
2. División 80/20 para entrenamiento/prueba
3. Experimentación con hiperparámetros
4. Evaluación de modelos

Importamos las librerias necesarias:

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import set_config
set_config(display="diagram")

Leemos el CSV:

In [ ]:
df = pd.read_csv("student_performance.csv")
print("Tamaño del dataset:", df.shape)

print("\nVista general del dataset:")
display(df.head())

print("\nInformacion general del dataset:")
print(df.info())

Tamaño del dataset: (1044, 17)

Vista general del dataset:


,sex,age,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,traveltime,studytime,failures,internet,romantic,goout,Walc,health,approved
0,F,18,GT3,A,4,4,at_home,teacher,2,2,0,no,no,4,1,3,0
1,F,17,GT3,T,1,1,at_home,other,1,2,0,yes,no,3,1,3,0
2,F,15,LE3,T,1,1,at_home,other,1,2,3,yes,no,2,3,3,1
3,F,15,GT3,T,4,2,health,services,1,3,0,yes,yes,2,1,5,1
4,F,16,GT3,T,3,3,other,other,1,2,0,no,no,2,2,5,1



Informacion general del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sex         1044 non-null   object
 1   age         1044 non-null   int64 
 2   famsize     1044 non-null   object
 3   Pstatus     1044 non-null   object
 4   Medu        1044 non-null   int64 
 5   Fedu        1044 non-null   int64 
 6   Mjob        1044 non-null   object
 7   Fjob        1044 non-null   object
 8   traveltime  1044 non-null   int64 
 9   studytime   1044 non-null   int64 
 10  failures    1044 non-null   int64 
 11  internet    1044 non-null   object
 12  romantic    1044 non-null   object
 13  goout       1044 non-null   int64 
 14  Walc        1044 non-null   int64 
 15  health      1044 non-null   int64 
 16  approved    1044 non-null   int64 
dtypes: int64(10), object(7)
memory usage: 138.8+ KB
None


Se separan los datos: 80% para entrenar y 20% para probar

In [ ]:
target_col = "approved"
X = df.drop(columns=[target_col])
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)
print(f"Entrenamiento: {X_train.shape}, Prueba: {X_test.shape}")

Entrenamiento: (835, 16), Prueba: (209, 16)


Identificamos las columnas nùmericas y categòricas:

In [ ]:
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

print("Numéricas:", num_cols)
print("Categóricas:", cat_cols)

Numéricas: ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'goout', 'Walc', 'health']
Categóricas: ['sex', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'internet', 'romantic']


Preprocesamiento:

In [ ]:
numeric_pipeline = Pipeline([("scaler", StandardScaler())])
categorical_pipeline = Pipeline([("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))])

preprocessor = ColumnTransformer([
    ("num", numeric_pipeline, num_cols),
    ("cat", categorical_pipeline, cat_cols)
], remainder="drop")

### **¿Qué es el Índice Gini?**
El índice Gini mide la **impureza** de un nodo. Representa la probabilidad de clasificar incorrectamente un elemento elegido aleatoriamente.

Se obtienen 5 àrboles de decisiòn con max_depth de 2, 4, 6, 8 y 10, con criterio "gini"

In [ ]:
depths = [2,4,6,8,10]
results_gini = []

for d in depths:
    clf = DecisionTreeClassifier(criterion="gini", max_depth=d, random_state=42)
    pipe = Pipeline([("preproc", preprocessor), ("clf", clf)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results_gini.append({"criterion":"gini", "max_depth": d, "accuracy": acc})
    print(f"[gini] max_depth={d} -> Accuracy: {acc:.4f}")

df_gini = pd.DataFrame(results_gini)
print("\nTabla de accuracies (criterio=\'gini\'):")
display(df_gini.sort_values("max_depth").reset_index(drop=True))

[gini] max_depth=2 -> Accuracy: 0.7799
[gini] max_depth=4 -> Accuracy: 0.7751
[gini] max_depth=6 -> Accuracy: 0.7799
[gini] max_depth=8 -> Accuracy: 0.7608
[gini] max_depth=10 -> Accuracy: 0.7129

Tabla de accuracies (criterio='gini'):


,criterion,max_depth,accuracy
0,gini,2,0.779904
1,gini,4,0.775120
2,gini,6,0.779904
3,gini,8,0.760766
4,gini,10,0.712919


### **¿Qué es la Entropía?**
La entropía mide el **desorden** o **incertidumbre** en un conjunto de datos. En teoría de información, representa la cantidad de información necesaria para describir una variable aleatoria.

Se obtienen 5 árboles de decisión con max_depth de 2, 4, 6, 8 y 10, usando el criterio "entropy"

In [ ]:
depths = [2, 4, 6, 8, 10]
results_entropy = []

for d in depths:
    clf = DecisionTreeClassifier(criterion="entropy", max_depth=d, random_state=42)
    pipe = Pipeline([("preproc", preprocessor), ("clf", clf)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results_entropy.append({"criterion": "entropy", "max_depth": d, "accuracy": acc})
    print(f"[entropy] max_depth={d} -> Accuracy: {acc:.4f}")

df_entropy = pd.DataFrame(results_entropy)
print("\nTabla de accuracies (criterio='entropy'):")
display(df_entropy.sort_values("max_depth").reset_index(drop=True))

[entropy] max_depth=2 -> Accuracy: 0.7799
[entropy] max_depth=4 -> Accuracy: 0.7895
[entropy] max_depth=6 -> Accuracy: 0.7847
[entropy] max_depth=8 -> Accuracy: 0.7560
[entropy] max_depth=10 -> Accuracy: 0.7321

Tabla de accuracies (criterio='entropy'):


,criterion,max_depth,accuracy
0,entropy,2,0.779904
1,entropy,4,0.789474
2,entropy,6,0.784689
3,entropy,8,0.755981
4,entropy,10,0.732057


8. hiperparámetros (max_depth y criterion) que permiten
obtener el árbol con mayor accuracy.

In [ ]:
all_results = results_gini + results_entropy
df_all = pd.DataFrame(all_results)

best_model = df_all.loc[df_all['accuracy'].idxmax()]

print("\nMejor configuración de hiperparámetros")

print(f"\nCriterio: {best_model['criterion']}")
print(f"Max Depth: {best_model['max_depth']}")
print(f"Accuracy: {best_model['accuracy']:.4f}")
print("\nTabla completa ordenada por accuracy:")
display(df_all.sort_values('accuracy', ascending=False).reset_index(drop=True))


Mejor configuración de hiperparámetros

Criterio: entropy
Max Depth: 4
Accuracy: 0.7895

Tabla completa ordenada por accuracy:


,criterion,max_depth,accuracy
0,entropy,4,0.789474
1,entropy,6,0.784689
2,gini,6,0.779904
3,gini,2,0.779904
4,entropy,2,0.779904
5,gini,4,0.775120
6,gini,8,0.760766
7,entropy,8,0.755981
8,entropy,10,0.732057
9,gini,10,0.712919


9. Variaciones en el hiperparámetro seleccionado.

In [ ]:
best_criterion = best_model['criterion']
best_depth = int(best_model['max_depth'])

min_samples_variations = [2, 10, 20]
results_min_samples = []

print(f"\nUsando criterion='{best_criterion}' y max_depth={best_depth}")
print(f"Experimentando con diferentes valores de min_samples_split:\n")

for min_samples in min_samples_variations:
    clf = DecisionTreeClassifier(
        criterion=best_criterion,
        max_depth=best_depth,
        min_samples_split=min_samples,
        random_state=42
    )
    pipe = Pipeline([("preproc", preprocessor), ("clf", clf)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    results_min_samples.append({
        "min_samples_split": min_samples,
        "accuracy": acc
    })

    print(f"min_samples_split={min_samples:2d} -> Accuracy: {acc:.4f}")

df_min_samples = pd.DataFrame(results_min_samples)





Usando criterion='entropy' y max_depth=4
Experimentando con diferentes valores de min_samples_split:

min_samples_split= 2 -> Accuracy: 0.7895
min_samples_split=10 -> Accuracy: 0.7895
min_samples_split=20 -> Accuracy: 0.7895


### Análisis:

El árbol de decisión **mantiene su exactitud** en 78.95% para todos los valores probados de min_samples_split.

**Explicación:**

El hiperparámetro min_samples_split no tuvo efecto porque con max_depth=4, incluso los nodos más profundos del árbol contienen aproximadamente 52 muestras, que es mayor que nuestro valor más restrictivo (20). Por lo tanto, todas las divisiones que el árbol intenta hacer son permitidas independientemente del valor de min_samples_split utilizado.

**Conclusión:**

En árboles poco profundos como el nuestro (depth=4), el hiperparámetro dominante es max_depth. El parámetro min_samples_split solo tendría impacto si se usaran profundidades mayores (depth > 6), donde los nodos terminales tendrían menos muestras y las restricciones de min_samples_split comenzarían a activarse.

In [ ]:
print("\nTabla de resultados:")
display(df_min_samples)

print("\nANÁLISIS DE RESULTADOS")

baseline_acc = best_model['accuracy']
print(f"\nAccuracy baseline (min_samples_split=2, por defecto): {baseline_acc:.4f}")

for i, row in enumerate(results_min_samples):
    min_samples = row['min_samples_split']
    acc = row['accuracy']
    diff = acc - baseline_acc

    if min_samples == 2:
        continue

    if diff > 0.001:
        status = "MEJORA"
        symbol = "↑"
    elif diff < -0.001:
        status = "EMPEORA"
        symbol = "↓"
    else:
        status = "SE MANTIENE"
        symbol = "→"

    print(f"\nmin_samples_split={min_samples}:")
    print(f"  Accuracy: {acc:.4f}")
    print(f"  Diferencia: {diff:+.4f} {symbol}")
    print(f"  Estado: {status}")



Tabla de resultados:


,min_samples_split,accuracy
0,2,0.789474
1,10,0.789474
2,20,0.789474



ANÁLISIS DE RESULTADOS

Accuracy baseline (min_samples_split=2, por defecto): 0.7895

min_samples_split=10:
  Accuracy: 0.7895
  Diferencia: +0.0000 →
  Estado: SE MANTIENE

min_samples_split=20:
  Accuracy: 0.7895
  Diferencia: +0.0000 →
  Estado: SE MANTIENE


## Conclusión Final

El desarrollo de este informe permitió explorar de manera práctica cómo diferentes configuraciones de modelos de aprendizaje automático influyen en su capacidad para predecir el desempeño académico de los estudiantes. A través del análisis realizado, fue posible identificar que el árbol de decisión con **criterion='entropy' y max_depth=4** proporcionó el mejor equilibrio entre complejidad y rendimiento, alcanzando un **accuracy del 78.95%** en el conjunto de prueba. Este resultado sugiere que una profundidad moderada permite capturar los patrones esenciales del dataset sin caer en sobreajuste.

Asimismo, la experimentación con el hiperparámetro min_samples_split mostró que, en este caso particular, su modificación no impactó el desempeño del modelo. Esto indica que la estructura general del árbol ya estaba bien regulada por otros parámetros y que los datos presentan relaciones lo suficientemente claras como para que este ajuste no sea determinante.

En conjunto, los resultados evidencian la importancia de evaluar distintos hiperparámetros y técnicas al construir modelos predictivos. También muestran que decisiones aparentemente pequeñas, como el criterio de división o la profundidad máxima del árbol, pueden tener un efecto directo en la calidad de las predicciones.
